In [163]:

import numpy as np 
import matplotlib.pyplot as plt 
import math
from scipy.spatial.transform import Rotation as R
from ipywidgets import interact, fixed
from itertools import product, combinations, permutations
from IPython.display import clear_output

#### Dynamical projection.

In [114]:
def norm_vector(n_, v_):
    p = n_ - np.dot(n_, v_.T) * v_
    return p

def rotate(x, y, z, vector):
    rx = np.array([[1, 0, 0], [0, np.cos(x), -np.sin(x)], [0, np.sin(x), np.cos(x)]])
    ry = np.array([[np.cos(y), 0, np.sin(y)], [0, 1, 0], [-np.sin(y), 0, np.cos(y)]])
    rz = np.array([[np.cos(z), -np.sin(z), 0], [np.sin(z), np.cos(z), 0], [0, 0, 1]])
    vector_ = np.dot(rz, np.dot(ry, np.dot(rx, vector)))
    
    return vector_ 

def angle_vector(vector1, vector2):
    dot_product = np.dot(vector1, vector2.T).T
    magnitude1 = np.linalg.norm(vector1, axis=1)
    magnitude2 = np.linalg.norm(vector2)
    angle = np.arccos(dot_product / (magnitude1 * magnitude2))
    angle_degrees = np.rad2deg(angle)
    return angle, angle_degrees

def angle_project_calc(v_plane, v1, v2):
    p1 = norm_vector(v1, v_plane)
    p2 = norm_vector(v2, v_plane)
    angle_project = angle_vector(p1, p2)
    return angle_project

def proj_coord(vector, r):
     #* Norm vector projecting on the viewpoint axis
    norm_111 = norm_vector(vector, np.array([[0,0,1]]))
    #* Angle between the norm vector and the viewpoint axis
    rho, _ = angle_vector(vector, np.array([[0,0,1]]))
    d = r*np.tan(rho/2)
    unit_len = np.sqrt(d**2/np.sum(norm_111**2, axis=1)).T
    norm_111_ = norm_111*unit_len
    x_len, y_len = norm_111_[:,0], norm_111_[:,1]
    
    return x_len, y_len

def rotation_axis(vector1, vector2):
    axis = np.cross(vector1, vector2)
    return axis

def euler_angle_calc(rotate_axis, rotate_degree):
    quaternion = R.from_rotvec(rotate_axis * rotate_degree / np.linalg.norm(rotate_axis))
    # quaternion_conj = R.from_rotvec(rotate_axis * -rotate_degree / np.linalg.norm(rotate_axis)) #* Minus rotation for vector buffer
    euler_angles = quaternion.as_euler('zyx',)
    # euler_angles_conj = quaternion_conj.as_euler('zyx',)
    
    return euler_angles

def inv_point(x, y, rot, lattice_vector):
    '''return the lattice point by clicking on the projection plane'''
    d = np.sqrt(x**2 + y**2)
    z = 2/(1+d**2)-1 #* cos(rho)
    x_true = np.sqrt((1-z**2)/(1+(y**2/x**2)))*np.sign(x)
    y_true = (y/x)*x_true
    
    convert_point = rot.apply(
        np.array([x_true, y_true, z]))@np.linalg.inv(lattice_vector)
    convert_point = convert_point/np.linalg.norm(convert_point)
    return str(np.round(convert_point, 3))
    
r = 1
x_010, y_100, symbol = [], [], []

#! Input the crystal information in the following format 
#* Modify it when necessary
#* Transfer the coordinates
a, b, c = [3.1, 3.1, 3.3]
alpha_rad = np.deg2rad(90)
beta_rad = np.deg2rad(90)
gamma_rad = np.deg2rad(120)

c31 = c * np.cos(beta_rad)
c32 = c * (np.cos(alpha_rad) - np.cos(beta_rad) * np.cos(gamma_rad)) / np.sin(gamma_rad)
c33 = c * np.sqrt(1 - np.cos(alpha_rad)**2 
                  - np.cos(beta_rad)**2 - np.cos(gamma_rad)**2
                  + 2 * np.cos(alpha_rad) * np.cos(beta_rad) * np.cos(gamma_rad)) \
                      / np.sin(gamma_rad)

lattice_vector = np.array([
    [a, 0, 0],
    [b * np.cos(gamma_rad), b * np.sin(gamma_rad), 0],
    [c31, c32, c33]
])

vector_buffer_raw = np.load(r'utils\vector_buffer_raw.npy')
longitude = np.load(r'utils\longitude.npy')
latitude = np.load(r'utils\latitude.npy')

vector_buffer = vector_buffer_raw.copy()
vector_buffer_ = vector_buffer@lattice_vector
vector_buffer_ = vector_buffer_/np.linalg.norm(vector_buffer_, axis=1)[:, None]

viewpoint_axis = np.array([[0,0,1]]) #* Origin viewpoint axis
viewpoint_axis_ = viewpoint_axis@lattice_vector
viewpoint_axis_ = viewpoint_axis_/np.linalg.norm(viewpoint_axis_)

special_vector = np.array([[1,1,1],[1,1,1]]) #* Special vector for displaying on projection plane
special_vector_ = special_vector@lattice_vector
special_vector_ = special_vector_/np.linalg.norm(special_vector_)

target_vector = np.array([[1,2,3]]) #* Target for rotating
target_vector_ = target_vector@lattice_vector
target_vector_ = target_vector_/np.linalg.norm(target_vector_)

rotate_axis = rotation_axis(viewpoint_axis_, target_vector_)
rotate_degree = angle_vector(viewpoint_axis_, target_vector_)[0]
euler_angle = euler_angle_calc(rotate_axis, rotate_degree)
# print(f'Euler angle for reaching target: {euler_angle_conj}')
# angle_x, angle_y, angle_z = np.deg2rad(
#     np.array([90, 49.744, 30.819])) #* Euler angle
rot = R.from_euler('zyx', euler_angle)
rot_conj = rot.inv()
# viewpoint_axis_true = rotate(angle_x, angle_y, angle_z, viewpoint_axis*(lattice_vector/7.66))
# vector_buffer_ = rotate(angle_x, angle_y, angle_z, vector_buffer_.T).T
vector_buffer_ = rot_conj.apply(vector_buffer_)
# special_vector_ = rotate(
#     angle_x, angle_y, angle_z, special_vector*(lattice_vector/7.66))
special_vector_ = rot_conj.apply(special_vector_)
    
#! Rotate the vector buffer to reach the target projection plane.
def main(vector_buffer_raw,
         special_vector_raw, 
         target_vector_raw,
         rot_raw,
         phi=0, theta=0, psi=0,
         x_=1e-3,y_=1e-3):
    
    y_100, x_010, symbol = [], [], []
    y_100_special, x_010_special, symbol_special = [], [], []
    
    euler_angle = np.array([phi,theta,psi])
    rot = R.from_euler('zyx', euler_angle, degrees=True)
    rot_inv = rot.inv()
    # rot_conj_inv = rot_conj.inv()
    vector_buffer_ = rot_inv.apply(vector_buffer_raw)
    special_vector_ = rot_inv.apply(special_vector_raw)
    target_vector_ = rot.apply(target_vector_raw)
    
    #* Click on plot
    rot_combineall = rot_raw*rot
    click_pole = inv_point(x_, y_, rot_combineall, lattice_vector)
    
    # for vector, vector_raw in zip(vector_buffer_, vector_buffer):
    #     x_len, y_len = proj_coord(vector, r)
        
    #     y_100.append(y_len)
    #     x_010.append(x_len)
    #     symbol.append(str(vector_raw))
        
    x_010, y_100 = proj_coord(vector_buffer_, r)
    symbol = [str(vector_raw) for vector_raw in vector_buffer]
        
    pole_list = np.array([x_010, y_100]).T
    valid_pole_ind = np.where(np.linalg.norm(pole_list, axis=1) <= 1.4)[0]
    valid_pole = pole_list[valid_pole_ind]

    # for vector, vector_raw in zip(special_vector_, special_vector):
    #     x_special, y_special = proj_coord(vector, r)
        
    #     y_100_special.append(y_special)
    #     x_010_special.append(x_special)
    #     symbol_special.append(str(vector_raw))
        
    x_010_special, y_100_special = proj_coord(special_vector_, r)
    symbol_special = [str(vector_raw) for vector_raw in special_vector]

    pole_list_special = np.array([x_010_special, y_100_special]).T
    valid_pole_ind_special = np.where(np.linalg.norm(pole_list_special, axis=1) <= 1.4)[0]
    valid_pole_special = pole_list_special[valid_pole_ind_special]

    fig, ax = plt.subplots(figsize=(20,20))
    
    plt.scatter(x_, y_, s=200, c='green', marker='o')
    plt.text(x_, y_, str(click_pole), fontsize=20)
    
    plt.scatter(valid_pole[:,0], valid_pole[:,1], s=100, c='r', marker='o')

    plt.scatter(valid_pole_special[:,0], valid_pole_special[:,1], s=200, c='b', marker='o')

    plt.axis('equal')
    plt.xlim(-1.4, 1.4)
    plt.ylim(-1.4, 1.4)
    for i in valid_pole_ind:
        plt.text(x_010[i], y_100[i], symbol[i], fontsize=10)
    for i in valid_pole_ind_special:
        plt.text(x_010_special[i], y_100_special[i], symbol_special[i], fontsize=20)

    #* Display the background 
    plt.scatter(longitude[:,0], longitude[:,1], 
                s=0.01, c='#838B8B', marker='o', zorder=0, alpha=0.2)
    plt.scatter(latitude[:,0], latitude[:,1], 
                s=0.01, c='#838B8B', marker='o', zorder=0, alpha=0.2)

    plt.show()

interact(main,
        vector_buffer_raw = fixed(vector_buffer_),
        special_vector_raw = fixed(special_vector_),
        target_vector_raw = fixed(target_vector_),
        rot_raw = fixed(rot),
        phi = (-180, 180, 2),
        theta = (-180, 180, 2),
        psi = (-180, 180, 2), 
        x_ = (-1.4, 1.4, 0.01),
        y_ = (-1.4, 1.4, 0.01),
)



interactive(children=(IntSlider(value=0, description='phi', max=180, min=-180, step=2), IntSlider(value=0, des…

<function __main__.main(vector_buffer_raw, special_vector_raw, target_vector_raw, rot_raw, phi=0, theta=0, psi=0, x_=0.001, y_=0.001)>

Let's step into the diffraction pattern.

The primary idea is:
- Define reciprocal lattice vector $a*, b*, c*$ at the first. Load the corresponding $(hkl)$ as well.
- When rotating the projection axis $[uvw]$, the reciprocal lattice points may rotate following the same rule refer to properties of the Fourier transform. Then use Weiss zone law to filter the plane.

In [ ]:
def norm_vector(n_, v_):
    p = n_ - np.dot(n_, v_.T) * v_
    return p

def rotate(x, y, z, vector):
    rx = np.array([[1, 0, 0], [0, np.cos(x), -np.sin(x)], [0, np.sin(x), np.cos(x)]])
    ry = np.array([[np.cos(y), 0, np.sin(y)], [0, 1, 0], [-np.sin(y), 0, np.cos(y)]])
    rz = np.array([[np.cos(z), -np.sin(z), 0], [np.sin(z), np.cos(z), 0], [0, 0, 1]])
    vector_ = np.dot(rz, np.dot(ry, np.dot(rx, vector)))
    
    return vector_ 

def angle_vector(vector1, vector2):
    dot_product = np.dot(vector1, vector2.T).T
    magnitude1 = np.linalg.norm(vector1, axis=1)
    magnitude2 = np.linalg.norm(vector2)
    angle = np.arccos(dot_product / (magnitude1 * magnitude2))
    angle_degrees = np.rad2deg(angle)
    return angle, angle_degrees

def angle_project_calc(v_plane, v1, v2):
    p1 = norm_vector(v1, v_plane)
    p2 = norm_vector(v2, v_plane)
    angle_project = angle_vector(p1, p2)
    return angle_project

def proj_coord(vector, r):
     #* Norm vector projecting on the viewpoint axis
    norm_111 = norm_vector(vector, np.array([[0,0,1]]))
    #* Angle between the norm vector and the viewpoint axis
    rho, _ = angle_vector(vector, np.array([[0,0,1]]))
    d = r*np.tan(rho/2)
    unit_len = np.sqrt(d**2/np.sum(norm_111**2, axis=1)).T
    norm_111_ = norm_111*unit_len
    x_len, y_len = norm_111_[:,0], norm_111_[:,1]
    
    return x_len, y_len

def rotation_axis(vector1, vector2):
    axis = np.cross(vector1, vector2)
    return axis

def euler_angle_calc(rotate_axis, rotate_degree):
    quaternion = R.from_rotvec(rotate_axis * rotate_degree / np.linalg.norm(rotate_axis))
    # quaternion_conj = R.from_rotvec(rotate_axis * -rotate_degree / np.linalg.norm(rotate_axis)) #* Minus rotation for vector buffer
    euler_angles = quaternion.as_euler('zyx',)
    # euler_angles_conj = quaternion_conj.as_euler('zyx',)
    
    return euler_angles

def inv_point(x, y, rot, lattice_vector):
    '''return the lattice point by clicking on the projection plane'''
    d = np.sqrt(x**2 + y**2)
    z = 2/(1+d**2)-1 #* cos(rho)
    x_true = np.sqrt((1-z**2)/(1+(y**2/x**2)))*np.sign(x)
    y_true = (y/x)*x_true
    
    convert_point = rot.apply(
        np.array([x_true, y_true, z]))@np.linalg.inv(lattice_vector)
    convert_point = convert_point/np.linalg.norm(convert_point)
    return str(np.round(convert_point, 3)), convert_point

def emission_point(uvw, length):
    '''  
    Return emission coordinate by [uvw] and 1/lambda=length to generate
    Ewald sphere
    '''
    u, v, w = uvw[0]
    x = np.sqrt(length**2/(1+v**2/u**2+w**2/u**2))
    y = v/u*x
    z = w/u*x
    
    return np.array([x,y,z])
    
r = 1
x_010, y_100, symbol = [], [], []

#! Input the crystal information in the following format 
#* Modify it when necessary
#* Transfer the coordinates
lattice_vector_real = np.array([3.1,3.1,3.3])
a, b, c = lattice_vector_real
alpha_rad = np.deg2rad(90)
beta_rad = np.deg2rad(90)
gamma_rad = np.deg2rad(120)

c31 = c * np.cos(beta_rad)
c32 = c * (np.cos(alpha_rad) - np.cos(beta_rad) * np.cos(gamma_rad)) / np.sin(gamma_rad)
c33 = c * np.sqrt(1 - np.cos(alpha_rad)**2 
                  - np.cos(beta_rad)**2 - np.cos(gamma_rad)**2
                  + 2 * np.cos(alpha_rad) * np.cos(beta_rad) * np.cos(gamma_rad)) \
                      / np.sin(gamma_rad)

lattice_vector = np.array([
    [a, 0, 0],
    [b * np.cos(gamma_rad), b * np.sin(gamma_rad), 0],
    [c31, c32, c33]
])

longitude = np.load(r'utils\longitude.npy')
latitude = np.load(r'utils\latitude.npy')

#* Radius of Ewald plane
r_ewald = 1/0.025079347455 #* Angstrom^-1, 200kV

#* Real space preparation
vector_buffer_raw = np.load(r'utils\vector_buffer_raw_alb2.npy')

vector_buffer = vector_buffer_raw.copy()
vector_buffer_ = vector_buffer@lattice_vector
vector_buffer_ = vector_buffer_/np.linalg.norm(vector_buffer_, axis=1)[:, None]

viewpoint_axis = np.array([[0,0,1]]) #* Origin viewpoint axis
viewpoint_axis_ = viewpoint_axis@lattice_vector
viewpoint_axis_ = viewpoint_axis_/np.linalg.norm(viewpoint_axis_)

special_vector = np.array([[1,1,1],[1,1,1]]) #* Special vector for displaying on projection plane
special_vector_ = special_vector@lattice_vector
special_vector_ = special_vector_/np.linalg.norm(special_vector_)

target_vector = np.array([[1,2,3]]) #* Target for rotating
target_vector_ = target_vector@lattice_vector
target_vector_ = target_vector_/np.linalg.norm(target_vector_)

rotate_axis = rotation_axis(viewpoint_axis_, target_vector_)
rotate_degree = angle_vector(viewpoint_axis_, target_vector_)[0]
euler_angle = euler_angle_calc(rotate_axis, rotate_degree)
rot = R.from_euler('zyx', euler_angle)
rot_conj = rot.inv()

vector_buffer_ = rot_conj.apply(vector_buffer_)
special_vector_ = rot_conj.apply(special_vector_)

#* Reciprocal space preparation
#* A first rotate will be applied as well
vector_buffer_hkl = np.load(r'utils\hkl_list\alb2_hkl_list.npy')

lattice_vector_inv = np.linalg.inv(lattice_vector)
vector_buffer_reci = vector_buffer_hkl.copy()
vector_buffer_reci_ = vector_buffer_reci@lattice_vector_inv.T

#* Return plane meet the Bragg condition
# inner_prod = vector_buffer_reci@target_vector.T
# bragg_condition = np.where(np.abs(inner_prod) < 1e-2)[0]
vector_buffer_reci_diff = rot_conj.apply(vector_buffer_reci_)

#! Rotate the vector buffer to reach the target projection plane.
def main(vector_buffer_raw,
         special_vector_raw, 
         target_vector_raw,
         rot_raw,
         vector_buffer_reci, #* Symbol -> reci
         vector_buffer_reci_diff, #* Coordinates -> reci
         phi=0, theta=0, psi=0,
         x_=1e-3,y_=1e-3):
    
    y_100, x_010, symbol = [], [], []
    y_100_special, x_010_special, symbol_special = [], [], []
    
    euler_angle = np.array([phi,theta,psi])
    rot = R.from_euler('zyx', euler_angle, degrees=True)
    rot_inv = rot.inv()
    # rot_conj_inv = rot_conj.inv()
    vector_buffer_ = rot_inv.apply(vector_buffer_raw)
    special_vector_ = rot_inv.apply(special_vector_raw)
    target_vector_ = rot.apply(target_vector_raw)
    
    rot_combineall = rot_raw*rot
    click_pole = inv_point(x_, y_, rot_combineall, lattice_vector)[0]
    central_axis = inv_point(1e-8, 1e-8, rot_combineall, lattice_vector)[1]
        
    x_010, y_100 = proj_coord(vector_buffer_, r)
    symbol = [str(vector_raw) for vector_raw in vector_buffer]
        
    pole_list = np.array([x_010, y_100]).T
    valid_pole_ind = np.where(np.linalg.norm(pole_list, axis=1) <= 1.4)[0]
    valid_pole = pole_list[valid_pole_ind]
        
    x_010_special, y_100_special = proj_coord(special_vector_, r)
    symbol_special = [str(vector_raw) for vector_raw in special_vector]

    pole_list_special = np.array([x_010_special, y_100_special]).T
    valid_pole_ind_special = np.where(np.linalg.norm(pole_list_special, axis=1) <= 1.4)[0]
    valid_pole_special = pole_list_special[valid_pole_ind_special]
    
    #* Diffraction pattern
    inner_prod_ = vector_buffer_reci@central_axis.reshape(-1,1)
    bragg_condition_ = np.where(np.abs(inner_prod_) < 1e-1)[0]
    #* Selection: bragg condition
    vector_buffer_reci_diff_ = rot_inv.apply(vector_buffer_reci_diff)
    vector_buffer_reci_diff_ = vector_buffer_reci_diff_[bragg_condition_] #* Coordinate
    vector_buffer_reci_ = vector_buffer_reci[bragg_condition_] #* Symbol
    
    #! Selection: Ewald sphere
    # emission_coord = emission_point(central_axis, r_ewald)
    emission_coord = np.array([0,0,r_ewald])
    emission_distance = np.linalg.norm(vector_buffer_reci_diff_-emission_coord, axis=1)
    
    valid_ind_1st = np.where(np.abs(emission_distance-r_ewald) <= 1e-2)[0]
    valid_ind_2nd = np.where(np.abs(emission_distance-r_ewald) <= 1.8e-2)[0]
    
    vector_buffer_reci_diff_1st, vector_buffer_reci_diff_2nd = vector_buffer_reci_diff_[valid_ind_1st], vector_buffer_reci_diff_[valid_ind_2nd]
    vector_buffer_reci_1st, vector_buffer_reci_2nd = vector_buffer_reci_[valid_ind_1st], vector_buffer_reci_[valid_ind_2nd]
    
    # print(vector_buffer_reci_, vector_buffer_reci_diff_)
    clear_output(wait=True)
    fig, (ax1, ax2) = plt.subplots(nrows=1, ncols=2, figsize=(40, 20))
    
    #* Real space
    ax1.scatter(x_, y_, s=200, c='green', marker='o')
    ax1.text(x_, y_, str(click_pole), fontsize=20)
    
    ax1.scatter(valid_pole[:,0], valid_pole[:,1], s=100, c='r', marker='o')

    ax1.scatter(valid_pole_special[:,0], valid_pole_special[:,1], s=200, c='b', marker='o')

    ax1.set_aspect('equal')
    ax1.set_xlim(-1.4, 1.4)
    ax1.set_ylim(-1.4, 1.4)
    
    for i in valid_pole_ind:
        ax1.text(x_010[i], y_100[i], symbol[i], fontsize=10)
    for i in valid_pole_ind_special:
        ax1.text(x_010_special[i], y_100_special[i], symbol_special[i], fontsize=20)

    #* Display the background 
    ax1.scatter(longitude[:,0], longitude[:,1], 
                s=0.01, c='#838B8B', marker='o', zorder=0, alpha=0.2)
    ax1.scatter(latitude[:,0], latitude[:,1], 
                s=0.01, c='#838B8B', marker='o', zorder=0, alpha=0.2)

    #* Reciprocal space
    ax2.set_facecolor('black')
    
    ax2.scatter(vector_buffer_reci_diff_1st[:,0], vector_buffer_reci_diff_1st[:,1],
                s=1500, c='w', marker='o')
        
    for i in range(len(vector_buffer_reci_1st)):
        vector_symbol = str(vector_buffer_reci_1st[i])
        ax2.text(vector_buffer_reci_diff_1st[:,0][i], vector_buffer_reci_diff_1st[:,1][i], 
                 vector_symbol, fontsize=20, color='r')

    ax2.scatter(vector_buffer_reci_diff_2nd[:,0], vector_buffer_reci_diff_2nd[:,1],
                s=700, c='w', marker='o')
    
    for i in range(len(vector_buffer_reci_2nd)):
        vector_symbol = str(vector_buffer_reci_2nd[i])
        ax2.text(vector_buffer_reci_diff_2nd[:,0][i], vector_buffer_reci_diff_2nd[:,1][i], 
                 vector_symbol, fontsize=20, color='r')
        
    ax2.set_aspect('equal')
    ax2.set_xlim(-2, 2)
    ax2.set_ylim(-2, 2)
    
    plt.title(f'{np.round(central_axis, 3)} zone axis', fontsize=20)
    plt.show()

interact(main,
        vector_buffer_raw = fixed(vector_buffer_),
        special_vector_raw = fixed(special_vector_),
        target_vector_raw = fixed(target_vector_),
        rot_raw = fixed(rot),
        vector_buffer_reci = fixed(vector_buffer_reci),
        vector_buffer_reci_diff = fixed(vector_buffer_reci_diff),
        phi = (-180, 180, 2),
        theta = (-180, 180, 2),
        psi = (-180, 180, 2), 
        x_ = (-1.4, 1.4, 0.01),
        y_ = (-1.4, 1.4, 0.01),
)